# Strands Introduction

Strands SDK is an open source python framework developed by AWS for building AI Agents. It is designed to simplify agent development by taking a model first approach. This allows developers to focus on defining goals and tools.

Strands enables the developer to call AWS Bedrock with minimal code. For tooling, Strands provides a common library `strands_tools` as well the ability to easily create tools. 

In [ ]:
from strands import Agent, tool
from strands_tools import calculator, current_time
@tool
def letter_counter(word: str, letter: str) -> int:
    """
    Count occurrences of a specific letter in a word.

    Args:
        word (str): The input word to search in
        letter (str): The specific letter to count

    Returns:
        int: The number of occurrences of the letter in the word
    """
    if not isinstance(word, str) or not isinstance(letter, str):
        return 0

    if len(letter) != 1:
        raise ValueError("The 'letter' parameter must be a single character")

    return word.lower().count(letter.lower())
agent = Agent(tools=[calculator, current_time, letter_counter])

# Ask the agent a question that uses the available tools
message = """
I have 3 requests for you to do:

1. What is the time right now?
2. Calculate 3111696 / 74088
3. Tell me how many letter R's are in the word "strawberry" 🍓
"""
agent(message)

## Find the list of community tools here
https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/community-tools-package/

# Local vs Cloud

In the above code, we can see with Strands we create an agents, but what is an agent? How does Amazon Bedrock play into Strand? In the above example the agent is provided a prompt and several tools. We can infer what the agent is doing, answering the prompt using the tools that we have provided. The question is __HOW does the the agent do this?__ 

How does this jupyter notebook and your local computer relate to Amazon Bedrock?

The majority of this workshop will focus on knowledge bases and tools and how they are used in conjunction with LLMs provided by Amazon bedrock. The LLM is running in Amazon Bedrock, and the knowledge base is being ingested by amazon bedrock, however the tools are not part of Bedrock.

See the following.

In [ ]:
## See your hostname
%%bash
echo $(hostname)

NV-Twalsh24


In [ ]:
from strands import Agent, tool
import socket

@tool
def host_name() -> str:
    """
    Gets the name of the host

    Returns:
        str: The name of the host
    """
    return socket.gethostname()

agent = Agent(tools=[host_name])

message = """
what is your the name of the host
"""
agent(message)

I'll get the name of the host for you.
Tool #1: host_name
The name of the host is **NV-Twalsh24**.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The name of the host is **NV-Twalsh24**.'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'host_name': ToolMetrics(tool={'toolUseId': 'tooluse_Y1C_Lf1nRjicT4NdYUzEEw', 'name': 'host_name', 'input': {}}, call_count=1, success_count=1, error_count=0, total_time=0.004027843475341797)}, cycle_durations=[1.3033673763275146], traces=[<strands.telemetry.metrics.Trace object at 0x00000140C2F37CB0>, <strands.telemetry.metrics.Trace object at 0x00000140C37E3100>], accumulated_usage={'inputTokens': 850, 'outputTokens': 67, 'totalTokens': 917}, accumulated_metrics={'latencyMs': 4874}), state={})

The preceding cells should have proved that the tool is being executed on the local machine. This means that the agent, is being executed by your local machine.

The implication of this is far reaching and beyond the scope of this workshop, but I felt that it was important to understand that *Strands isn't used to build models or do magic. Strands enables us to leverage the features of Amazon Bedrock in order to create agents that use AI models.*

# Case Study

The remainder of this demo is done in the form of a case study and designed around using a key feature of Bedrock, __Knowledge Bases__.
- *A __Knowledge Base__ is trusted data source enables the AI Agents to provide answers and completing tasks.*

The Federal government requires states to make Wildlife harvest records publicly available. Each State does this on their own Wildlife website making it extremely hard to build a tool that can scrape them reliably. We want to create an Agent that can scrape these sites reliably and upload the data to a knowledge base. We will then create an agent that references that knowledge base in answering questions.

### Goals
1. Given a web page search for PDF files 
2. Upload the pdf files to s3
    - Keep the bucket organized and follow the following naming convention `<animal>/<state>/<year>.pdf`
3. Create a knowledge base
4. Create an agent that references that knowledge base
5. Do not leverage use_aws tool

### Examples
- https://wildlife.utah.gov/hunting/main-hunting-page/big-game/big-game-harvest-data.html
- https://huntillinois.org/harvest-data

#### Use AWS tool
Amazon provides a tool, `use_aws`, that should make this job very easy in theory. In practice this did not work. 

In my first attempt, I tried creating 2 tools to enforce naming consistency and then letting `use_aws` run with it. I knew this solution would not be optimal, but I wanted to see if it would work. The agent could not succeed at this, but I learned quite alot about `use_aws`. My personal recommendation is to avoid this to outside of POC work, it is extremely greedy in how it opperates, deploying resources, failing and not cleaning up after its self. It will continue try and fail and try something new that will fail until it runs out of tokens. This is a great way to run up costs and lower the security of your aws environment. 

This attempt was unable to successfully upload pdf files in every attempt. I would create "pdfs" in s3 but they would be empty files with no contents

## Design Considerations
This agent is designed to only do 1 workflow.
- Find PDF files on a website
- Download the PDF files
- Upload PDFs to S3 
- Create/update the knowledge base

When we think about this workflow, the only AI part of this work is identifying the PDF file urls un a website. Further we can assume that this is going to effectively amount to parsing html. `<a href="<pdf file link>">`
- Government websites especially for hunting can often be behind the times so vanilla html is not uncommon.
- Strands provides a tool `http_request` which we can leverage for these vanilla sites
- More modern Javascript webpages may be more difficult to parse, there are other Strands tools that could improve this in theory, but I couldn't get them configured to run in the time I had. 

The remaining workflow is algorithmic, so instead of leaving it up to the agent we can build tooling for this. Further, we want this to be a singular tool so that there is only 1 possible entry point for the agent to call. If we provided multiple tools we would need to add more instructions (probably numbered instructions) to enforce the order of opperations, but it would be hard to ensure that if a step failed the agent would not do the step after that.


In [ ]:
## you can provide a different bedrock model id here if you want
model = "us.anthropic.claude-sonnet-4-20250514-v1:0"
## provide your name, but keep it short, no more then 6 characters
user ="twalsh" 
region = "us-east-1"

#### Creating a tool
It's time to create a tool for the AI agent too leverage. The empty function `ingest_files_and_create_knowledge_base` will be the tool that we want the agent to use. 

The function definition and the parameters have been defined

Your objectives are as follows
- Fill out the tool document comment so that the agent can effectively use the tool
- Fill out the contents of the loop in the tools root function so that it calls create_knowledge_base correctly
- Fill out create_knowledge_base function to execute the calls to the necessary knowledge_base management functions

Every area that must be filled out is labelled

In [ ]:
from strands import tool
import requests
import os
from knowledge_base_management import create_knowledge_base_with_s3_vectors, retrieve_knowledge_base, update_knowledge_base_with_s3_vectors
os.environ["BYPASS_TOOL_CONSENT"] = "true"

def download_pdf(url, year):
    response =  requests.get(url)
    local_file_name = year+'.pdf'
    if response.status_code == 200:
        with open(local_file_name, 'wb') as file:
            file.write(response.content)
        print(f"PDF downloaded successfully: {local_file_name}")
        return local_file_name
    else:
        print(f"Failed to download PDF. Status code: {response.status_code}")

def create_knowledge_base(files, topic= f"hunting-{user}"):
    ## Fill this in Referncing functions in knowledge_base_management.py
    kb_id =  retrieve_knowledge_base(topic)
    if kb_id is None:
        return create_knowledge_base_with_s3_vectors(topic, files, region)
    else:
        return update_knowledge_base_with_s3_vectors(topic, files, kb_id, region)

@tool
def ingest_files_and_create_knowledge_base(animal:str, state:str, file_details: dict[str, str]) :
    """
    Downloads PDF files from provided URLs, stores them in an S3 bucket, and creates or updates a vector-based Amazon Bedrock knowledge base.

    This tool performs the following steps:
    ###################################### REPLACE THIS LINE AND FILL THIS IN########################################
    1. Generates a unique name for each file.
    2. Uploads the files to an S3 bucket (creates the bucket if it doesn't exist).
    3. Creates or updates a Bedrock knowledge base using the uploaded files.

    Parameters:
    ###################################### REPLACE THIS LINE AND FILL THIS IN########################################
        animal (str): The type of animal the data is related to (e.g., "deer").
        state (str): The U.S. state the data is associated with (e.g., "Illinois").
        file_details (dict[str, str]): A dictionary of years and urls:
            - key (str): The key should be the year the data in the file represents.
            - value (str): The value should be the direct link to a downloadable PDF file.

    Returns:
    ###################################### REPLACE THIS LINE AND FILL THIS IN########################################
        str: The ID of the created or updated knowledge base.

    Example:
    ###################################### REPLACE THIS LINE AND FILL THIS IN########################################
        ingest_files_and_create_knowledge_base(
            animal="deer",
            state="Illinois",
            file_details=[
                {"2022", "https://example.com/deer-report-2022.pdf"},
                {"2023", "https://example.com/deer-report-2023.pdf"}
            ]
        )

    """ 
    ## Fill loop here 
    files = []
    for file in file_details.items():
        year, url = file
        source = download_pdf(url, year)
        target = f"{animal.lower()}/{state.lower()}/{year}.pdf"
        files.append((source,target))
    kb_id = create_knowledge_base(files)
    ## at the end of this function we need to write the kb_id to an environment variable so that it can be referenced later in the jupyter notebook
    os.environ["BEDROCK_KB_ID"] = kb_id
    return kb_id


### The Web Scraping Agent
Fill in the System Prompt and provide the tooling so that the Web Scraping agent will be 

In [ ]:
from strands import Agent
from strands_tools import http_request
import os
os.environ["BYPASS_TOOL_CONSENT"] = "true"


system_prompt = """
You are Hunting data scraping agent, your job is to fetch hunting data from a given web page and create knowledge bases from the PDFs on the web page.
[Instructions]
- you will be provided a URL to a states hunting website and an animal
- Use HTTP GET to load the web page for the url provided
- Identify the state that the webpage is for
- Identify all the pdf files from the page for the animal that is provided. Pdf files should have a particular year in which they contain data for
- If a year is presented as a range (e.g. 2022-2023) then that you should treat the year as the beginning of the range
- If there are multiple pdf files for a single year and animal then choose one pdf for that year
- If there are multiple pdf files for a single year and animal pick the file that is for the general season if possible
- If there are multiple pdf files for a single year and animal but you are unable to identify a general season file then pick the first file file that animal and year
- Using the tool you have been provided
    - Ingest all the files that you identified and generate knowledge base name with the provided animal name and the state that you identified
    - If this tool fails stop and tell the user there is a problem
"""

hunting_kb_agent = Agent(
    tools=[ingest_files_and_create_knowledge_base, http_request ],
    system_prompt = system_prompt,
    model=model
)

response = hunting_kb_agent("Get the Deer Data from https://wildlife.utah.gov/hunting/main-hunting-page/big-game/big-game-harvest-data.html")


### Using the Knowledge Base

Strands provides us with the tool `retrieve` inorder to tell our agent to use the knowledge base as reference. The agent must be provided with the Knowledge Base's Id. If the default AWS region is not set on your aws account or the region of the knowledge base is not the default then you will need to provide the region in the system prompt.

- In the following cell you have been provded with a base system prompt
- Create an agent that references the knowledge base with this system prompt
- Then modify the system prompt to answer the the user prompt below


In [ ]:
from strands import Agent
from strands_tools import retrieve
import os
os.environ["BYPASS_TOOL_CONSENT"] = "true"
kb_id = os.environ["BEDROCK_KB_ID"]
system_prompt = f"""
You are Hunting guide, your clients will ask you all about hunting. Do not answer questions unrelated to hunting. 
[Instructions]
- Search the knowledge base (ID: {kb_id}) in the region {region} and answer questions based on that knowledge base. That knowledge base contains data on Utah and Illinois.
- If you encounter an error accessing the knowledge base print it out to the user
"""
guide_agent = Agent(
    system_prompt = system_prompt,
    tools=[retrieve],
    model=model
)

response = guide_agent("Historically what regions of Utah have the highest success rates for archery hunters?")


In [ ]:
## Update the system prompt to prevent this question from being answered
response = guide_agent("What color is taylor swifts hair?")

In [ ]:
response = guide_agent("What compare the color of a deer's coat to taylor swifts hair?")

## Lets load some more data
We need to update the Agent tooling to support updating the knowledge base if it already exists.
Go back up to the the cell where you created the tooling four our web scraping agent and make the following updates
- Update the tools name say suggest it updates the knowledge base
- Update the tools description to say that it updates the knowledge base
- Update the tool to check AWS to see if the KB exists (This ability is provided in knowledge_base_management.py)
- Update the logic so that it will update the knowledge base if it exists
- Recreate the agent (this needs to be done because you have renamed the tool)

__Disclaimer__: Illinois transitioned from vanilla html to JS while I was working on this, the web scraping agent still works, but I have had it fail once or twice, you may need to rerun the agent if it fails

In [ ]:
from strands import tool
import requests
import os
from knowledge_base_management import create_knowledge_base_with_s3_vectors, retrieve_knowledge_base, update_knowledge_base_with_s3_vectors
os.environ["BYPASS_TOOL_CONSENT"] = "true"

def download_pdf(url, year):
    response =  requests.get(url)
    local_file_name = year+'.pdf'
    if response.status_code == 200:
        with open(local_file_name, 'wb') as file:
            file.write(response.content)
        print(f"PDF downloaded successfully: {local_file_name}")
        return local_file_name
    else:
        print(f"Failed to download PDF. Status code: {response.status_code}")

def create_or_update_knowledge_base(files, topic= f"hunting-workshop-{user}"):
    kb_id =  retrieve_knowledge_base(topic)
    if kb_id is None:
        return create_knowledge_base_with_s3_vectors(topic, files, region)
    else:
        return update_knowledge_base_with_s3_vectors(topic, files, kb_id, region)

@tool
def ingest_files_and_create_or_update_knowledge_base(animal:str, state:str, file_details: dict[str, str]) :
    """
    Downloads PDF files from provided URLs, stores them in an S3 bucket, and creates or updates a vector-based Amazon Bedrock knowledge base.

    This tool performs the following steps:
    1. Generates a unique name for each file.
    2. Uploads the files to an S3 bucket (creates the bucket if it doesn't exist).
    3. Creates or updates a Bedrock knowledge base using the uploaded files.

    Parameters:
        animal (str): The type of animal the data is related to (e.g., "deer").
        state (str): The U.S. state the data is associated with (e.g., "Illinois").
        file_details (dict[str, str]): A dictionary of years and urls:
            - key (str): The key should be the year the data in the file represents.
            - value (str): The value should be the direct link to a downloadable PDF file.

    Returns:
        str: The ID of the created or updated knowledge base.

    Example:
        ingest_files_and_create_knowledge_base(
            animal="deer",
            state="Illinois",
            file_details=[
                {"2022", "https://example.com/deer-report-2022.pdf"},
                {"2023", "https://example.com/deer-report-2023.pdf"}
            ]
        )

    """ 
    ## Fill loop here 
    files = []
    for file in file_details.items():
        year, url = file
        source = download_pdf(url, year)
        target = f"{animal.lower()}/{state.lower()}/{year}.pdf"
        files.append((source,target))
    kb_id = create_or_update_knowledge_base(files)
    ## at the end of this function we need to write the kb_id to an environment variable so that it can be referenced later in the jupyter notebook
    os.environ["BEDROCK_KB_ID"] = kb_id
    return kb_id


In [ ]:
from strands import Agent
from strands_tools import http_request
import os
os.environ["BYPASS_TOOL_CONSENT"] = "true"


system_prompt = """
You are Hunting data scraping agent, your job is to fetch hunting data from a given web page and create knowledge bases from the PDFs on the web page.
[Instructions]
- you will be provided a URL to a states hunting website and an animal
- Use HTTP GET to load the web page for the url provided
- Identify the state that the webpage is for
- Identify all the pdf files from the page for the animal that is provided. Pdf files should have a particular year in which they contain data for
- If a year is presented as a range (e.g. 2022-2023) then that you should treat the year as the beginning of the range
- If there are multiple pdf files for a single year and animal then choose one pdf for that year
- If there are multiple pdf files for a single year and animal pick the file that is for the general season if possible
- If there are multiple pdf files for a single year and animal but you are unable to identify a general season file then pick the first file file that animal and year
- Using the tool you have been provided
    - Ingest all the files that you identified and generate knowledge base name with the provided animal name and the state that you identified
    - If this tool fails stop and tell the user there is a problem
"""

hunting_kb_agent = Agent(
    tools=[ingest_files_and_create_or_update_knowledge_base, http_request ],
    system_prompt = system_prompt,
    model=model
)


In [ ]:
response = hunting_kb_agent("Get the Deer Data from https://huntillinois.org/harvest-data")

## Lets add memory and the calculator

Strands has a tools for making agents more robust.
- recreate the guide agent with memory and calculator
- create user prompts to leverage those tools

https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/community-tools-package/

Hint: you shouldn't need to update the system prompt, but you can

In [ ]:
from strands import Agent
from strands_tools import retrieve, memory, calculator
import os
os.environ["BYPASS_TOOL_CONSENT"] = "true"

os.environ["BEDROCK_KB_ID"] = kb_id
system_prompt = f"""
You are Hunting guide, your clients will ask you all about hunting. Do not answer questions unrelated to hunting.
[Instructions]
- Search the knowledge base (ID: {kb_id}) in the region us-east-1 and answer questions based on that knowledge base. That knowledge base contains data on Utah and Illinois.
- If you encounter an error accessing the knowledge base print it out to the user
"""


guide_agent = Agent(
    system_prompt = system_prompt,
    tools=[retrieve, calculator, memory],
    model=model
)


In [ ]:
## ask a question

In [ ]:
## ask a related question that proves the agent uses memory

In [ ]:
## ask a question that forces our agent to use the calculator

# Making an agent capable of searching the internet

Duck duck go provides us with a python library. 
- Create a tool to Leverage the `ddgs.text()` function to preform a keyword search 
- Provide the agent the ability to use the tool 
- Explain to the agent how to leverage tool in the system prompt
- Limit the number of searches the agent can do (this is optional, but annoying if you do not)

In [ ]:
import logging
from strands import Agent, tool
from strands_tools import calculator
from ddgs import DDGS

logging.basicConfig(level=logging.INFO)

@tool
def websearch(
    keywords: str,
    region: str = "us-en",
    max_results: int | None = None,
) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=10)
        return results if results else "No results found."
    except Exception as e:
        return f"Exception: {e}"

os.environ["BYPASS_TOOL_CONSENT"] = "true"

os.environ["BEDROCK_KB_ID"] = kb_id
system_prompt = f"""
You are Hunting guide, your clients will ask you all about hunting. Do not answer questions unrelated to hunting.
[Instructions]
1. Search the knowledge base (ID: {kb_id}) in the region us-east-1 and answer questions based on that knowledge base.
    - If you encounter an error accessing the knowledge base print it out to the user
###################################### REPLACE THIS LINE AND update this prompt to add search instructions########################################
2. If you do not find information in your knowledge base preform a web search to answer
    - For keywords reference the state and animal you were asked about and look at .gov and .org results
    - Search at most 1 time
"""
# Create a Strands agent
better_hunting_guide = Agent(
    name="smart hunting guide",
    system_prompt= system_prompt,
    tools=[retrieve,memory, calculator, websearch],
    callback_handler=None,
    model=model
)

In [ ]:

response = better_hunting_guide("How many deer are hunted in Wisconsin annually?")

# Future add ons

I wanted to add an agent to agent call where the Second Agent tells the first Agent to load Wisconsin's data to the knowledge base. Some of the issues that I was running into are interesting
- not all of the government sites with statistics are .gov 
- Internet searching to find exact source that I want to is a challenge for the agent
- because the first agent can't be counted on to find the proper source how do we solve loading data that may be less reliable to our knowledge base?
